

---

###  Food Delivery Data Integration – Notebook Introduction

In this notebook, three different data files are combined into one dataset.
The files include order details, user information, and restaurant data.
These files are merged using common ID columns to form a single final dataset.
The merged file is named **final_food_delivery_dataset.csv** and is used for all analysis.
Using Python, we load the data, join the tables, and check the results.
This dataset is then used to answer questions related to sales, users, and restaurants.


In [1]:
# Step 1: Load CSV Data (orders)
import pandas as pd

orders = pd.read_csv("/orders.csv")


In [2]:
# Step 2: Load JSON Data (users)
users = pd.read_json("/users.json")


In [3]:
# Step 3: Load SQL Data (restaurants)
import sqlite3

conn = sqlite3.connect(":memory:")

with open("/restaurants.sql", "r") as f:
    sql_script = f.read()

conn.executescript(sql_script)

restaurants = pd.read_sql("SELECT * FROM restaurants", conn)


In [4]:
# Step 4: Merge Orders with Users (LEFT JOIN)
orders_users = orders.merge(
    users,
    on="user_id",
    how="left"
)


In [5]:
# Step 5: Merge with Restaurants (LEFT JOIN)
final_df = orders_users.merge(
    restaurants,
    on="restaurant_id",
    how="left"
)


In [6]:
# Step 6: Rename duplicate columns (if needed)
final_df.rename(columns={
    "city_x": "user_city",
    "city_y": "restaurant_city"
}, inplace=True)


In [7]:
# Step 7: Export Final Dataset
final_df.to_csv("final_food_delivery_dataset.csv", index=False)


 final_food_delivery_dataset.csv

In [8]:
import pandas as pd

df = pd.read_csv("/content/final_food_delivery_dataset.csv")

# convert date
df["order_date"] = pd.to_datetime(df["order_date"], format="%d-%m-%Y")


Multiple Choice Questions

Which city has the highest total revenue (total_amount) from Gold members?  

In [9]:
gold_city_revenue = (
    df[df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
)

gold_city_revenue.sort_values(ascending=False)


,total_amount
city,
Chennai,1080909.79
Pune,1003012.32
Bangalore,994702.59
Hyderabad,896740.19


Which cuisine has the highest average order value across all orders?

In [10]:
cuisine_avg = df.groupby("cuisine")["total_amount"].mean()
cuisine_avg.sort_values(ascending=False)


,total_amount
cuisine,
Mexican,808.021344
Italian,799.448578
Indian,798.466011
Chinese,798.389020


How many distinct users placed orders worth more than ₹1000 in total (sum of all their orders)?

In [11]:
user_total = df.groupby("user_id")["total_amount"].sum()

count_users = (user_total > 1000).sum()
count_users


np.int64(2544)

Which restaurant rating range generated the highest total revenue?

In [12]:
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ["3.0–3.5", "3.6–4.0", "4.1–4.5", "4.6–5.0"]

df["rating_range"] = pd.cut(df["rating"], bins=bins, labels=labels, include_lowest=True)

df.groupby("rating_range")["total_amount"].sum().sort_values(ascending=False)


/tmp/ipython-input-3381336125.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("rating_range")["total_amount"].sum().sort_values(ascending=False)


,total_amount
rating_range,
4.6–5.0,2197030.75
3.0–3.5,2136772.70
4.1–4.5,1960326.26
3.6–4.0,1717494.41


Among Gold members, which city has the highest average order value?

In [13]:
gold_city_avg = (
    df[df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .mean()
)

gold_city_avg.sort_values(ascending=False)


,total_amount
city,
Chennai,808.459080
Hyderabad,806.421034
Bangalore,793.223756
Pune,781.162243


Which cuisine has the lowest number of distinct restaurants but still contributes significant revenue?

In [14]:
cuisine_stats = df.groupby("cuisine").agg(
    restaurant_count=("restaurant_id", "nunique"),
    revenue=("total_amount", "sum")
)

cuisine_stats.sort_values("restaurant_count")


,restaurant_count,revenue
cuisine,,
Chinese,120,1930504.65
Indian,126,1971412.58
Italian,126,2024203.80
Mexican,128,2085503.09


What percentage of total orders were placed by Gold members? (Rounded to nearest integer)

In [15]:
total_orders = len(df)
gold_orders = len(df[df["membership"] == "Gold"])

percent = round((gold_orders / total_orders) * 100)
percent


50

Which restaurant has the highest average order value but less than 20 total orders?

In [16]:
rest_stats = df.groupby("restaurant_name_x").agg(
    avg_order=("total_amount", "mean"),
    order_count=("order_id", "count")
)

filtered = rest_stats[rest_stats["order_count"] < 20]
filtered.sort_values("avg_order", ascending=False).head()


,avg_order,order_count
restaurant_name_x,,
Hotel Dhaba Multicuisine,1040.222308,13
Sri Mess Punjabi,1029.180833,12
Ruchi Biryani Punjabi,1002.140625,16
Sri Delights Pure Veg,989.467222,18
Classic Kitchen Family Restaurant,973.167895,19


Which combination contributes the highest revenue?

In [17]:
combo = df.groupby(["membership", "cuisine"])["total_amount"].sum()
combo.sort_values(ascending=False)


membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

During which quarter of the year is the total revenue highest?

In [18]:
df["quarter"] = df["order_date"].dt.to_period("Q")

df.groupby("quarter")["total_amount"].sum().sort_values(ascending=False)


,total_amount
quarter,
2023Q3,2037385.10
2023Q4,2018263.66
2023Q1,1993425.14
2023Q2,1945348.72
2024Q1,17201.50


Numerical Answers

How many total orders were placed by users with Gold membership?

In [19]:
gold_orders = df[df["membership"] == "Gold"]
len(gold_orders)


4987

What is the total revenue (rounded to nearest integer) generated from orders placed in Hyderabad city?

In [20]:
hyderabad_revenue = df[df["city"] == "Hyderabad"]["total_amount"].sum()
round(hyderabad_revenue)


1889367

How many distinct users placed at least one order?

In [21]:
df["user_id"].nunique()


2883

What is the average order value (rounded to 2 decimals) for Gold members?

In [22]:
gold_avg = df[df["membership"] == "Gold"]["total_amount"].mean()
round(gold_avg, 2)


np.float64(797.15)

How many orders were placed for restaurants with rating ≥ 4.5?

In [23]:
high_rating_orders = df[df["rating"] >= 4.5]
len(high_rating_orders)


3374

How many orders were placed in the top revenue city among Gold members only?

In [24]:
# find top city
top_city = (
    df[df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .idxmax()
)

# count orders in that city (Gold only)
gold_top_city_orders = df[
    (df["membership"] == "Gold") &
    (df["city"] == top_city)
]

len(gold_top_city_orders)


1337

Fill in the blanks

The column used to join orders.csv and users.json is __________.

In [46]:
join_column_orders_users = list(set(orders.columns) & set(users.columns))
print("1️⃣ Join column (orders & users):", join_column_orders_users)


1️⃣ Join column (orders & users): ['user_id']


The dataset containing cuisine and rating information is stored in __________ format.

In [41]:
# 2. Dataset containing cuisine and rating information format
# -----------------------------
print("2️⃣ Cuisine & rating dataset format: SQL")



2️⃣ Cuisine & rating dataset format: SQL


The total number of rows in the final merged dataset is __________.

In [27]:
len(df)


10000

If a user has no matching record in users.json, the merged values will be __________.

In [40]:
df[df["membership"].isna()].head()



,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating,rating_range,quarter


The Pandas function used to combine datasets based on a key is __________.

In [42]:
# 5. Pandas function used to combine datasets
# -----------------------------
print("5️⃣ Pandas function used for joining: merge()")


5️⃣ Pandas function used for joining: merge()


The column membership in the final dataset originates from the __________ file.  

In [43]:
# 6. Source of 'membership' column
# -----------------------------
membership_source = "users.json" if "membership" in users.columns else "unknown"
print("6️⃣ 'membership' column originates from:", membership_source)

6️⃣ 'membership' column originates from: users.json


The join key used to combine orders data with restaurant details is __________.  

In [47]:
join_column_orders_restaurants = list(set(orders.columns) & set(restaurants.columns))
print("7️⃣ Join key (orders & restaurants):", join_column_orders_restaurants)


7️⃣ Join key (orders & restaurants): ['restaurant_id', 'restaurant_name']


The column that helps identify the type of food served by a restaurant is __________.  

In [44]:
# 8. Column identifying type of food
# -----------------------------
food_type_column = "cuisine" if "cuisine" in restaurants.columns else "unknown"
print("8️⃣ Column identifying food type:", food_type_column)

8️⃣ Column identifying food type: cuisine


If a user places multiple orders, their personal details appear __________ times in the final merged dataset.  

In [45]:
# 9. Effect of multiple orders by same user
# -----------------------------
user_order_counts = final_df["user_id"].value_counts()
multiple_orders_effect = "multiple times" if user_order_counts.max() > 1 else "once"
print("9️⃣ User personal details appear:", multiple_orders_effect)

9️⃣ User personal details appear: multiple times




---

###  Conclusion

In this project, different data files were successfully combined into one final dataset.
By merging orders, users, and restaurant details, we created a single source of information for analysis.
This helped in understanding customer behavior, popular cities, and high-performing cuisines.
The final dataset made it easy to calculate revenue, order patterns, and membership impact.
Overall, this process shows how raw data from different sources can be connected to get meaningful insights.
